# Menjelajahi Privasi Diferensial

Proyek pembelajaran mesin biasanya melibatkan proses berulang analisis data untuk mendapatkan wawasan tentang data dan menentukan variabel mana yang paling mungkin membantu membangun model prediktif. Menganalisis data biasanya melibatkan fungsi agregat dan statistik yang memberikan wawasan tentang distribusi statistik variabel dan hubungan di antara keduanya. Dengan volume data yang besar, agregasi memberikan tingkat abstraksi; tetapi dengan jumlah data yang lebih kecil, atau dengan analisis berulang, bahkan hasil agregasi dapat mengungkapkan detail tentang masing-masing pengamatan.

*Privasi diferensial* adalah teknik yang dirancang untuk menjaga privasi masing-masing poin data dengan menambahkan "derau" ke data. Tujuannya adalah untuk memastikan bahwa derau yang cukup ditambahkan untuk memberikan privasi bagi masing-masing nilai sambil memastikan bahwa susunan statistik keseluruhan data tetap konsisten, dan agregasi menghasilkan hasil yang serupa secara statistik seperti ketika digunakan dengan data mentah asli.

## Menginstal SDK SmartNoise

[*SmartNoise*](https://smartnoise.org/) adalah kotak alat dari OpenDP; sebuah proyek gabungan antara peneliti di Microsoft, Harvard University, dan kontributor lain yang bertujuan untuk menyediakan blok penyusun untuk menggunakan privasi diferensial dalam analisis data dan proyek pembelajaran mesin.

> **Catatan**: SmartNoise sedang dalam tahap awal pengembangan saat ini.

Mari kita mulai dengan menginstal paket SDK Python SmartNoise. Anda dapat mengabaikan kesalahan apa pun tentang kompatibilitas Azure CLI dalam latihan ini.

In [ ]:
pip install opendp-smartnoise==0.1.4.2

## Memuat data

Sekarang, mari kita lihat beberapa data mentah. Dalam kasus ini, kita memiliki 10.000 catatan untuk pasien yang telah diperiksa untuk diabetes.

In [ ]:
import pandas as pd

data_path = 'data/diabetes.csv'
diabetes = pd.read_csv(data_path)
diabetes.describe()

Output dari kode di atas menunjukkan statistik ringkasan utama untuk variabel dalam himpunan data diabetes.

## Melakukan analisis

Anda dapat menggunakan SmartNoise untuk membuat analisis yang menambahkan derau ke data sumber. Matematika yang mendasari cara derau ditambahkan dapat menjadi sangat kompleks, tetapi SmartNoise menangani sebagian besar detail untuk Anda. Tetapi, ada beberapa konsep yang berguna untuk diperhatikan.

- **Batas atas dan bawah**: *Clamping* digunakan untuk mengatur batas atas dan bawah pada nilai untuk variabel. Ini diperlukan untuk memastikan bahwa derau yang dihasilkan oleh SmartNoise konsisten dengan distribusi yang diharapkan dari data asli.
**Ukuran sampel**: Untuk menghasilkan data privat berbeda yang konsisten untuk beberapa agregasi, SmartNoise perlu mengetahui ukuran sampel data yang akan dihasilkan.
- **Epsilon**: Sederhananya, *epsilon* adalah nilai non-negatif yang memberikan ukuran kebalikan dari jumlah derau yang ditambahkan ke data. Epsilon yang rendah menghasilkan himpunan data dengan tingkat privasi yang lebih besar, sedangkan epsilon yang tinggi menghasilkan himpunan data yang lebih mendekati data asli. Umumnya, Anda harus menggunakan nilai epsilon antara 0 dan 1. Epsilon berkorelasi dengan nilai lain bernama *delta*, yang menunjukkan peluang bahwa laporan yang dihasilkan oleh analisis tidak sepenuhnya privat.

Dengan mengingat konsep-konsep ini, periksa dan jalankan kode berikut, yang membuat analisis dan melaporkan nilai **Usia** rata-rata dari data privat yang berbeda. Nilai rata-rata aktual dari data mentah asli juga ditampilkan untuk perbandingan.

In [ ]:
import opendp.smartnoise.core as sn

cols = list(diabetes.columns)
age_range = [0.0, 120.0]
samples = len(diabetes)

with sn.Analysis() as analysis:
    # load data
    data = sn.Dataset(path=data_path, column_names=cols)
    
    # Convert Age to float
    age_dt = sn.to_float(data['Age'])
    
    # get mean of age
    age_mean = sn.dp_mean(data = age_dt,
                          privacy_usage = {'epsilon': .50},
                          data_lower = age_range[0],
                          data_upper = age_range[1],
                          data_rows = samples
                         )
    
analysis.release()

# print differentially private estimate of mean age
print("Private mean age:",age_mean.value)

# print actual mean age
print("Actual mean age:",diabetes.Age.mean())

## Menjelajahi distribusi data dengan histogram

Merupakan hal yang umum ketika menganalisis data untuk memeriksa distribusi variabel menggunakan histogram.

Sebagai contoh, mari kita lihat distribusi usia yang sebenarnya dalam himpunan data diabetes.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

ages = list(range(0, 130, 10))
age = diabetes.Age

# Plot a histogram with 10-year bins
n_age, bins, patches = plt.hist(age, bins=ages, color='blue', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('True Age Distribution')
plt.show()
print(n_age.astype(int))

Sekarang mari kita bandingkan histogram Usia privat yang berbeda.

In [ ]:
import matplotlib.pyplot as plt

with sn.Analysis() as analysis:
    data = sn.Dataset(path = data_path, column_names = cols)

    age_histogram = sn.dp_histogram(
            sn.to_int(data['Age'], lower=0, upper=120),
            edges = ages,
            upper = 10000,
            null_value = -1,
            privacy_usage = {'epsilon': 0.5}
        )
    
analysis.release()

plt.ylim([0,7000])
width=4
agecat_left = [x + width for x in ages]
agecat_right = [x + 2*width for x in ages]
plt.bar(list(range(0,120,10)), n_age, width=width, color='blue', alpha=0.7, label='True')
plt.bar(agecat_left, age_histogram.value, width=width, color='orange', alpha=0.7, label='Private')
plt.legend()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

print(age_histogram.value)

Histogram cukup mirip untuk memastikan bahwa laporan berdasarkan data privat yang berbeda memberikan wawasan yang sama seperti laporan dari data mentah.

## Menghitung kovarian

Tujuan umum lainnya dari analisis adalah untuk membangun hubungan antar variabel. SmartNoise menyediakan fungsi *kovarian* privat yang berbeda yang dapat membantu dalam hal ini.

In [ ]:
with sn.Analysis() as analysis:
    sn_data = sn.Dataset(path = data_path, column_names = cols)

    age_bp_cov_scalar = sn.dp_covariance(
                left = sn.to_float(sn_data['Age']),
                right = sn.to_float(sn_data['DiastolicBloodPressure']),
                privacy_usage = {'epsilon': 1.0},
                left_lower = 0.,
                left_upper = 120.,
                left_rows = 10000,
                right_lower = 0.,
                right_upper = 150.,
                right_rows = 10000)
analysis.release()
print('Differentially private covariance: {0}'.format(age_bp_cov_scalar.value[0][0]))
print('Actual covariance', diabetes.Age.cov(diabetes.DiastolicBloodPressure))

Dalam kasus ini, kovarian antara **Usia** dan **DisatolicBloodPressure** adalah positif, yang menunjukkan bahwa pasien yang lebih tua cenderung memiliki tekanan darah yang lebih tinggi.

## Menggunakan kueri SQL

Selain fungsionalitas **Analisis**, SmartNoise memungkinkan Anda menggunakan kueri SQL terhadap sumber data untuk mengambil hasil agregat privat yang berbeda.

Pertama, Anda perlu menentukan metadata untuk tabel dalam skema data Anda. Anda dapat melakukan hal ini di file .yml, seperti file **diabetes.yml** di folder **/metadata**. Metadata menjelaskan bidang dalam tabel, termasuk tipe data dan nilai minimum dan maksimum untuk bidang numerik.

In [ ]:
from opendp.smartnoise.metadata import CollectionMetadata

meta = CollectionMetadata.from_file('metadata/diabetes.yml')
print (meta)

Dengan metadata yang ditentukan, Anda dapat membuat *pembaca* yang dapat Anda kueri. Dalam contoh berikut, kita akan membuat **PandasReader** untuk membaca data mentah dari kerangka data Pandas, dan **PrivateReader** yang menambahkan lapisan privasi diferensial ke **PandasReader**.

In [ ]:
from opendp.smartnoise.sql import PandasReader, PrivateReader

reader = PandasReader(diabetes, meta)
private_reader = PrivateReader(reader=reader, metadata=meta, epsilon_per_column=0.7)
print('Readers ready.')

Sekarang Anda dapat mengirimkan kueri SQL yang mengembalikan kumpulan hasil agregat ke pembaca privat.

In [ ]:
query = 'SELECT Diabetic, AVG(Age) AS AvgAge FROM diabetes.diabetes GROUP BY Diabetic'

result_dp = private_reader.execute(query)
print(result_dp)

Mari kita bandingkan hasilnya dengan agregasi yang sama dari data mentah.

In [ ]:
result = reader.execute(query)
print(result)

Anda dapat menyesuaikan perilaku **PrivateReader** dengan **parameter epsilon_per_column**.

Mari kita coba pembaca dengan nilai epsilon (privasi rendah) tinggi, dan satu lagi dengan nilai epsilon (privasi tinggi) rendah.

In [ ]:
low_privacy_reader = PrivateReader(reader, meta, 5.0)  # large epsilon, less privacy
result = low_privacy_reader.execute(query)
print(result)
print()

high_privacy_reader = PrivateReader(reader, meta, 0.1)  # smaller epsilon, more privacy
result = high_privacy_reader.execute(query)
print(result)

Perhatikan bahwa hasil pembaca epsilon (privasi rendah) tinggi lebih mendekati hasil sebenarnya dari data mentah daripada hasil dari pembaca epsilon (privasi tinggi) rendah.

## Mempelajari selengkapnya

Untuk mempelajari selengkapnya tentang privasi diferensial dengan SmartNoise, lihat [https://smartnoise.org](https://smartnoise.org/)